---
# Load dataset
---

In [51]:
# If you print some element from `dataset_map['train'][element_index]['input_ids']` you'll see that lots of element
vect = [ele[ele.nonzero()].size(0) for ele in dataset_mag_map['train'][:]['input_ids']]

/home/vivoli/miniconda3/envs/arxiv-manipulation/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370156314/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  


In [52]:
max_vect = max(vect)
min_vect = min(vect)
sum_vect = sum(vect)
len_vect = len(vect)

print(f" max: {max_vect} \n min: {min_vect} \n avg: {sum_vect/len_vect}")

 max: 512 
 min: 2 
 avg: 162.88874527462752


In [53]:
print(dataset_mag_map['train'][:10])

{'input_ids': tensor([[  102, 11261,   669,  ...,     0,     0,     0],
        [  102,  6773,   165,  ...,     0,     0,     0],
        [  102,   833,   111,  ...,     0,     0,     0],
        ...,
        [  102,  5005,  4994,  ...,     0,     0,     0],
        [  102,   121,   238,  ...,     0,     0,     0],
        [  102, 15794,   190,  ...,     0,     0,     0]])}


---
# Train with Trainer
---

From [here](https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py) you can get an idea from were the code has been borrowed.

In [54]:
# Data collator
# This one will take care of randomly masking the tokens.
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

train_dataset = dataset_mag_map['train']
eval_dataset = dataset_mag_map['valid']

# Inizialize TrainerArguments
training_args = TrainingArguments(
    output_dir=output_dir,           # [def.`tmp_trainer`] output directory
    num_train_epochs=3,              # [def.   3 ] total # of training epochs
    per_device_train_batch_size=16,   # [def.   8 ] batch size per device during training
    per_device_eval_batch_size=16,    # [def.   8 ] batch size for evaluation
    evaluation_strategy="no",        # [def. 'no'] evaluation is done (and logged) every eval_steps
    warmup_steps=0,                  # [def.   0 ] number of warmup steps for learning rate scheduler
    weight_decay=0,                  # [def.   0 ] strength of weight decay 
    learning_rate=5e-4,              # [def. 5e-5] 
    logging_dir='./logs',            # [def. runs/__id__] directory for storing logs. TensorBoard log directory.
)

# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)



In [55]:
# Training

checkpoint = None
train_result = trainer.train(resume_from_checkpoint=checkpoint)
trainer.save_model()  # Saves the tokenizer too for easy upload
metrics = train_result.metrics

# max_train_samples = dataset_args.max_train_samples if dataset_args.max_train_samples is not None else len(train_dataset)
max_train_samples = len(train_dataset)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))

trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()



Step,Training Loss
500,3.358500
1000,3.659500
1500,3.709500
2000,3.710400
2500,3.653800
3000,3.688000
3500,3.567900
4000,3.496200
4500,3.408400
5000,3.282000


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [56]:
# Evaluation

logger.info("*** Evaluate ***")

metrics = trainer.evaluate()

# max_val_samples = dataset_args.max_val_samples if dataset_args.max_val_samples is not None else len(eval_dataset)
max_val_samples = len(eval_dataset)
metrics["eval_samples"] = min(max_val_samples, len(eval_dataset))

import math
perplexity = math.exp(metrics["eval_loss"])
metrics["perplexity"] = perplexity

trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

In [57]:
wandb.finish()

train/loss,2.4244
train/learning_rate,2e-05
train/epoch,3.0
train/global_step,13491
_runtime,6519
_timestamp,1618932242
_step,27
train/train_runtime,5491.643
train/train_samples_per_second,2.457
train/total_flos,7.290962682789888e+16
eval/loss,2.33759


train/loss,▆█████▇▇▆▆▅▅▅▄▄▃▃▃▂▂▂▂▁▁▁▁
train/learning_rate,██▇▇▇▇▆▆▆▅▅▅▅▄▄▄▄▃▃▃▂▂▂▂▁▁
train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
_runtime,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
_timestamp,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
train/train_runtime,▁
train/train_samples_per_second,▁
train/total_flos,▁
eval/loss,▁
